In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu, wilcoxon

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.metric.idtxl_pid import bivariate_pid_3D
from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage
from mesostat.utils.signals import bin_data
from mesostat.utils.pandas_helper import pd_append_row
from mesostat.stat.permtests import percentile_twosided, perm_test_resample


from lib.sych.data_fc_db_raw import DataFCDatabase
from lib.sych.plot_helper import imshow_dataset_by_mouse, imshow_dataset_by_session
from lib.sych.metric_helper import metric_by_session, metric_by_selector

%load_ext autoreload
%autoreload 2

Appended root directory /home/alyosha/work/git/pub-2020-exploratory-analysis


In [2]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
#params['root_path_data'] = gui_fpath('h5path', './')

In [3]:
dataDB = DataFCDatabase(params)

Searching for data files


In [10]:
ds = DataStorage('sych_result_higher_order.h5')

In [5]:
mc = MetricCalculator(serial=False, verbose=False, nCore=4)

Redirecting STDOUT to log_21990.txt
Redirecting STDOUT to log_21991.txt
Redirecting STDOUT to log_21992.txt
Redirecting STDOUT to log_21993.txt
Resumed STDOUT
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21990.txt
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21991.txt
Redirecting STDOUT to log_21992.txt
Redirecting STDOUT to log_21993.txt
Redirecting STDOUT to log_21990.txt
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21991.txt
Resumed STDOUT
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21992.txt
Redirecting STDOUT to log_21993.txt
Redirecting STDOUT to log_21990.txt
Resumed STDOUT
Redirecting STDOUT to log_21991.txt
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21992.txt
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21993.txt
Redirecting STDOUT to log_21990.txt
Redirecting STDOUT to log_21991.txt
Redirecting STDOUT to log_21992.txt
Resumed STDOUT
Resumed STDOUT
Redirecting STDOUT to log_21991.txt
Resumed STDOUT
Resumed ST

# Hypotheses

* H1) Feedforwards Prefrontal
    - Sources={M1, S1};   Targets={ PrL, LO, VO, M2 или Cg1}
* H2) High order is Sensory/Motor Thalamus
    - Sources={M1, S1};   Targets={PO, VM} 
* H3) Thalamus as source
    - Sources={VPM, PO};  Targets={S1, S2}
* H4) Motor Thalamus synchronization
    - Sources={VM, VL, LD};    Targets={M1,M2}

In [6]:
def calc_pid(dataDB, sources, targets, mousename, datatype, zscoreDim, cropTime, trialType, performance):
    channelNames = dataDB.get_channel_labels(mousename)
    sourceIdxs = [channelNames.index(s) for s in sources]
    targetIdxs = [channelNames.index(t) for t in targets]
    
    dataLst = dataDB.get_neuro_data({'mousename' : mousename}, datatype=datatype,
                                    zscoreDim=zscoreDim, cropTime=cropTime,
                                    trialType=trialType, performance=performance)
    
    data = np.concatenate(dataLst, axis=0)
    data = np.mean(data, axis=1)  # Average out time
    data = bin_data(data, 4, axis=1)
    
    settings_estimator = {'pid_estimator': 'TartuPID', 'lags_pid': [0, 0]}
    
    mc.set_data(data, 'rp')
    rez = mc.metric3D('BivariatePID', '',
                      metricSettings={'settings_estimator' : settings_estimator,
                                      'src' : sourceIdxs},
                      sweepSettings={'trg' : targetIdxs})
    
    # Since all channels are binned to the same quantiles,
    # the permutation test is exactly the same for all of them, so we need any three channels as input
    settings_test = {'src' : [0, 1], 'trg' : 2, 'settings_estimator' : settings_estimator}
    fTest = lambda x: bivariate_pid_3D(x, settings_test)
    dataTest = data[:, :3][..., None]  # Add fake 1D sample dimension
    
    fRand = perm_test_resample(fTest, dataTest, 1000, iterAxis=1)
    
    rezTest = [
        percentile_twosided(fTrue, fRand, settings={"haveEffectSize" : True, "haveMeans" : True})
        for fTrue in rez
    ]
    
    rezTest = np.array(rezTest)
    df = pd.DataFrame(columns=['mousename', 'S1', 'S2', 'T', 'PID', 'p', 'effSize', 'muTrue', 'muRand'])
    for iTrg, trgName in enumerate(targets):
        for iType, infType in enumerate(['U1', 'U2', 'red', 'syn']):
            rowLst = [mousename, *sources, trgName, infType, *rezTest[iTrg, 1:, iType]]
            df = pd_append_row(df, rowLst, skip_repeat=False)
                
    return df

In [7]:
# H1
sources = ['M1_l', 'S1_bf']
targets = ['PrL', 'LO', 'VO', 'M2', 'Cg1']

rezDF1 = pd.DataFrame()
for mousename in dataDB.mice:
    dfTmp = calc_pid(dataDB, sources, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    rezDF1 = rezDF1.append(dfTmp)

Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

...

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDOUT to log_21958.txt
Resumed STDOUT
Redirecting STDO

In [8]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rezDF1.sort_values(by=['S1', 'S2', 'T', 'PID', 'mousename']))

,mousename,S1,S2,T,PID,p,effSize,muTrue,muRand
16,mvg_4,M1_l,S1_bf,Cg1,U1,0.001,47.596536,2.372204e-01,0.007380
16,mvg_7,M1_l,S1_bf,Cg1,U1,0.001,198.507041,9.792637e-01,0.007588
16,mvg_8,M1_l,S1_bf,Cg1,U1,0.001,48.275126,1.632286e-01,0.005081
16,mvg_9,M1_l,S1_bf,Cg1,U1,0.009,3.637813,3.357836e-02,0.009782
17,mvg_4,M1_l,S1_bf,Cg1,U2,0.399,0.002679,7.195345e-03,0.007183
17,mvg_7,M1_l,S1_bf,Cg1,U2,1.000,-1.482770,-2.423373e-10,0.007242
17,mvg_8,M1_l,S1_bf,Cg1,U2,0.001,4.295407,1.845889e-02,0.004757
17,mvg_9,M1_l,S1_bf,Cg1,U2,0.978,-1.388504,1.127180e-03,0.009782
18,mvg_4,M1_l,S1_bf,Cg1,red,0.001,469.303678,9.719008e-01,0.004040
18,mvg_7,M1_l,S1_bf,Cg1,red,0.001,39.705972,8.897641e-02,0.004198


In [12]:
ds.save_dataframe('H1', rezDF1, {})

In [14]:
ds.list_dsets_pd()

AttributeError: 'Group' object has no attribute 'shape'

In [ ]:
# H2
sources = ['M1_l', 'S1_bf']
targets = ['Po', 'VM']

rezDF2 = pd.DataFrame()
for mousename in dataDB.mice:
    dfTmp = calc_pid(dataDB, sources, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    rezDF2 = rezDF1.append(dfTmp)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rezDF1.sort_values(by=['S1', 'S2', 'T', 'PID', 'mousename']))

In [ ]:
# H3
sources = ['Po', 'VPM']
targets = ['S1_bf', 'S2']

rezDF3 = pd.DataFrame()
for mousename in dataDB.mice:
    dfTmp = calc_pid(dataDB, sources, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    rezDF3 = rezDF3.append(dfTmp)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rezDF3.sort_values(by=['S1', 'S2', 'T', 'PID', 'mousename']))

In [ ]:
# H4
sources1 = ['VM', 'VL']
sources2 = ['VM', 'LDVL']
sources3 = ['VL', 'LDVL']
targets = ['M1_l', 'M2']

rezDF4 = pd.DataFrame()
for mousename in dataDB.mice:
    dfTmp1 = calc_pid(dataDB, sources1, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    dfTmp2 = calc_pid(dataDB, sources2, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    dfTmp3 = calc_pid(dataDB, sources3, targets, mousename, 'bn_session', None, [3, 3.5], 'iGO', 'expert')
    
    rezDF4 = rezDF4.append(dfTmp1)
    rezDF4 = rezDF4.append(dfTmp2)
    rezDF4 = rezDF4.append(dfTmp3)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(rezDF4.sort_values(by=['S1', 'S2', 'T', 'PID', 'mousename']))